## Running the applet:

- First we need to select the data from the server in a new csv file with all the data we need. 
    - sessio_number
    - rawDataLeft
    - rawDataRight
    - created
- These will help out when selecting the data on back4app
- Then we export the csv file and we put it within the folder of this app.
- The next step is to make a json file using the `make_json(file_to_read, file_to_make)`
- After the json file is made (we can actually keep it in memory by returning the dictionary) we read the file
- We then extract the urls (read the json file for more info)
- Then using the urls we can download the data
- Last piece is to rename the files in the correct format.


In [1]:
# Python program to read json file
import csv
import json
import os
# Request url info
import requests  

In [ ]:
def download_files(url_list, download_path):
    '''
    Download files into provided pathway given a list of url

    @Params:
    url_list (list) - List of url to download from
    download_path (str) - path to download the files into

    Does not have return values
    '''

    # Save text from url as a .txt
    for url in url_list:
        f = requests.get(url)
        
        # filename gotten by splitting url and removing the unique ID of each dataset
        filename = url.split('/')[-1][33:-4] + '.txt'

        # Save files as txt
        with open(download_path + filename, "w+") as text_file:
            text_file.write(f.text)

    return


def make_json(csvFilePath, jsonFilePath):
    # create a dictionary
    data = {}
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
            # Assuming a column named 'No' to
            # be the primary key
            key = rows['sessionNumber']
            data[key] = rows
            data[key]['rawDataLeft'] = json.loads(rows['rawDataLeft'])
            data[key]['rawDataRight'] = json.loads(rows['rawDataRight'])
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))


def read_json(jsonFilePath):
    with open(jsonFilePath, "r") as read_it:
        data = json.load(read_it)

    return data

    
def rename_files(path):
    print(path)
    files = os.listdir(path)  # List of files in the data path folder
    for file in files:
        filename = file.split(".txt")[0]
        str_list = filename.split("_")
        new_filename = str_list[2] + "_" + str_list[3] + "_" + str_list[1] + "_" + str_list[0] + ".txt"
        os.rename(os.path.join(path, file), os.path.join(path, new_filename))



## Main Code

In [47]:
make_json('Recent json file/RawDataFiles.csv', 'result.json')
data = read_json('result.json')

urls = []
for d in data.values():
    urls.append(d['rawDataLeft']['url']) 
    urls.append(d['rawDataRight']['url'])

download_path = 'download/'
download_files(urls, download_path)
rename_files('download')